In [31]:
import datetime as dt
import yfinance as yf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

### MACD-moving average convergence divergence

In [8]:
tickers = ['600600.SS','600132.SS']
all_data = {}

In [9]:
for ticker in tickers:
    temp = yf.download(ticker,period='6mo',interval = '1d')
    temp.dropna(how='any',inplace=True)
    all_data[ticker]=temp

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [10]:
all_data

{'600600.SS':                   Open        High         Low       Close   Adj Close  \
 Date                                                                     
 2021-06-24  107.000000  107.680000  104.320000  105.300003  104.394333   
 2021-06-25  105.190002  110.849998  104.480003  110.680000  109.728050   
 2021-06-28  109.830002  113.879997  108.690002  112.250000  111.284554   
 2021-06-29  112.680000  116.680000  111.510002  115.809998  114.813927   
 2021-06-30  115.809998  118.110001  113.160004  115.650002  114.655312   
 ...                ...         ...         ...         ...         ...   
 2021-12-20   99.629997  100.269997   98.720001   99.050003   99.050003   
 2021-12-21   99.300003  103.260002   99.000000  102.269997  102.269997   
 2021-12-22  102.269997  103.589996  101.320000  102.050003  102.050003   
 2021-12-23  102.000000  104.199997   99.800003  101.440002  101.440002   
 2021-12-24  102.430000  105.949997  101.309998  105.089996  105.089996   
 
          

In [14]:
def MACD(DF,a=12,b=26,c=9):
    df = DF.copy()
    df['ma_fast'] = df['Adj Close'].ewm(span=a,min_periods = a).mean()
    df['ma_slow'] = df['Adj Close'].ewm(span=b,min_periods = b).mean()
    df['MACD'] = df['ma_fast']-df['ma_slow']
    df['signal'] = df['MACD'].ewm(span=c,min_periods = c).mean()
    
    return df.loc[:,['MACD','signal']]

In [15]:
for ticker in all_data:
    all_data[ticker][['MACD','signal']] = MACD(all_data[ticker])

### ATR

In [20]:
def ATR(DF,n=14):
    df = DF.copy()
    df['H-L'] = df['High']-df['Low'] #high minus low
    df['H-PC'] = df['High']-df['Adj Close'].shift(1) #high minus previous close
    df['L-PC'] = df['Low']-df['Adj Close'].shift(1)  #low minus previous close
    df['TR'] = df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].ewm(span=n,min_periods=n).mean()
    
    return df['ATR']

In [21]:
for ticker in all_data:
    all_data[ticker]['ATR'] = ATR(all_data[ticker])

### Bollinger Band

In [26]:
def Boll(DF,n=14):
    df = DF.copy()
    df['MB'] = df['Adj Close'].rolling(n).mean() #simple moving average for the middle band
    df['UB'] = df['MB'] + 2*df['Adj Close'].rolling(n).std(ddof=0)
    df['LB'] = df['MB'] - 2*df['Adj Close'].rolling(n).std(ddof=0)
    df['BB_width'] = df['UB']-df['LB']
    
    return df[['UB','MB','LB','BB_width']]

In [28]:
for ticker in all_data:
    all_data[ticker][['UB','MB','LB','BB_width']] = Boll(all_data[ticker],n=20)

### RSI

In [33]:
def RSI(DF,n=14):
    df = DF.copy()
    df['change'] = df['Adj Close']-df['Adj Close'].shift(1)
    df['gain'] = np.where(df['change']>=0,df['change'],0)
    df['loss'] = np.where(df['change']<0,-1*df['change'],0)
    df['avg_gain'] = df['gain'].ewm(alpha=1/n,min_periods=n).mean()
    df['avg_loss'] = df['loss'].ewm(alpha=1/n,min_periods=n).mean()
    df['rs'] = df['avg_gain']/df['avg_loss']
    df['rsi'] = 100 - (100 / (1+df['rs']))
    
    return df['rsi']
    

In [34]:
for ticker in all_data:
    all_data[ticker]['rsi'] = RSI(all_data[ticker],n=14)

### Renko

In [36]:
import talib


In [42]:
close = np.random.random(100)
output = talib.SMA(close,timeperiod=14)

In [43]:
help(talib.SMA)

Help on function SMA in module talib._ta_lib:

SMA(...)
    SMA(real[, timeperiod=?])
    
    Simple Moving Average (Overlap Studies)
    
    Inputs:
        real: (any ndarray)
    Parameters:
        timeperiod: 30
    Outputs:
        real



In [44]:
output

array([       nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan, 0.50075669, 0.48481323,
       0.53497791, 0.54239246, 0.4970038 , 0.49048098, 0.44881577,
       0.41030133, 0.35618315, 0.38592527, 0.4047321 , 0.43663321,
       0.47944002, 0.46747315, 0.47185099, 0.51775714, 0.44875315,
       0.4625276 , 0.468571  , 0.47595829, 0.52483715, 0.56045103,
       0.5867376 , 0.57789898, 0.51262925, 0.4948892 , 0.4589148 ,
       0.50802843, 0.49670376, 0.50097391, 0.56359734, 0.56031174,
       0.58046685, 0.55739678, 0.5503065 , 0.5749609 , 0.59349928,
       0.55673362, 0.59060543, 0.56544618, 0.58700161, 0.54828346,
       0.53630145, 0.54928613, 0.53370812, 0.57950781, 0.59342352,
       0.5759731 , 0.57190973, 0.53664107, 0.49331234, 0.54222228,
       0.57542783, 0.563742  , 0.56486154, 0.57003557, 0.60191213,
       0.54858049, 0.53314357, 0.48758624, 0.47222157, 0.50455